In [1]:
! pip install sqlalchemy
! pip install PyMySQL

In [2]:
import pymysql
import seaborn as sns
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import scipy.stats as stats
import warnings

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import getpass  # to get the password without showing the input
password = getpass.getpass()

········


In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

In [6]:
data10 = pd.read_sql_query(""" select r.rental_id, f.film_id, fc.category_id, c.name, r.rental_date, r.return_date, f.title, f.description, f.release_year, f.rental_duration, f.rental_rate, f.rating, f.length
from rental r
join inventory i
on i.inventory_id = r.inventory_id
join film f
on f.film_id = i.film_id
join film_category fc
on fc.film_id = f.film_id
join category c
on c.category_id = fc.category_id
order by rental_date;

""", engine)

In [7]:
data10

,rental_id,film_id,category_id,name,rental_date,return_date,title,description,release_year,rental_duration,rental_rate,rating,length
0,1,80,8,Family,2005-05-24 22:53:30,2005-05-26 22:04:30,BLANKET BEVERLY,A Emotional Documentary of a Student And a Gir...,2006,7,2.99,G,148
1,2,333,12,Music,2005-05-24 22:54:33,2005-05-28 19:40:33,FREAKY POCUS,A Fast-Paced Documentary of a Pastry Chef And ...,2006,7,2.99,R,126
2,3,373,3,Children,2005-05-24 23:03:39,2005-06-01 22:12:39,GRADUATE LORD,A Lacklusture Epistle of a Girl And a A Shark ...,2006,7,2.99,G,156
3,4,535,11,Horror,2005-05-24 23:04:41,2005-06-03 01:43:41,LOVE SUICIDES,A Brilliant Panorama of a Hunter And a Explore...,2006,6,0.99,R,181
4,5,450,3,Children,2005-05-24 23:05:21,2005-06-02 04:33:21,IDOLS SNATCHERS,A Insightful Drama of a Car And a Composer who...,2006,5,2.99,NC-17,84
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16040,13780,385,5,Comedy,2006-02-14 15:16:03,NaT,GROUNDHOG UNCUT,A Brilliant Panorama of a Astronaut And a Tech...,2006,6,4.99,PG-13,139
16041,11847,388,5,Comedy,2006-02-14 15:16:03,NaT,GUNFIGHT MOON,A Epic Reflection of a Pastry Chef And a Explo...,2006,5,0.99,NC-17,70
16042,15423,388,5,Comedy,2006-02-14 15:16:03,NaT,GUNFIGHT MOON,A Epic Reflection of a Pastry Chef And a Explo...,2006,5,0.99,NC-17,70
16043,15314,604,5,Comedy,2006-02-14 15:16:03,NaT,MULAN MOON,A Emotional Saga of a Womanizer And a Pioneer ...,2006,4,0.99,G,160


In [8]:
data10.dtypes

rental_id                   int64
film_id                     int64
category_id                 int64
name                       object
rental_date        datetime64[ns]
return_date        datetime64[ns]
title                      object
description                object
release_year                int64
rental_duration             int64
rental_rate               float64
rating                     object
length                      int64
dtype: object

In [9]:
data_num = data10.select_dtypes(include = np.number)
data_cat = data10.select_dtypes(include = 'object')
cat_data = pd.get_dummies(data10, drop_first=True) # if you need to save the encoding info, this won't do
cat_data

rental_id  film_id  category_id         rental_date  \
0              1       80            8 2005-05-24 22:53:30   
1              2      333           12 2005-05-24 22:54:33   
2              3      373            3 2005-05-24 23:03:39   
3              4      535           11 2005-05-24 23:04:41   
4              5      450            3 2005-05-24 23:05:21   
...          ...      ...          ...                 ...   
16040      13780      385            5 2006-02-14 15:16:03   
16041      11847      388            5 2006-02-14 15:16:03   
16042      15423      388            5 2006-02-14 15:16:03   
16043      15314      604            5 2006-02-14 15:16:03   
16044     150048        1            6 2021-10-27 13:20:51   

              return_date  release_year  rental_duration  rental_rate  length  \
0     2005-05-26 22:04:30          2006                7         2.99     148   
1     2005-05-28 19:40:33          2006                7         2.99     126   
2     2005-06-01 22:12:39          2006                7         2.99     156   
3     2005-06-03 01:43:41          2006                6         0.99     181   
4     2005-06-02 04:33:21          2006                5         2.99      84   
...                   ...           ...              ...          ...     ...   
16040                 NaT          2006                6         4.99     139   
16041                 NaT          2006                5         0.99      70   
16042                 NaT          2006                5         0.99      70   
16043                 NaT          2006                4         0.99     160   
16044 2021-10-21 00:00:00          2006                6         0.99      86   

       name_Animation  name_Children  name_Classics  name_Comedy  \
0                   0              0              0            0   
1                   0              0              0            0   
2                   0              1              0            0   
3                   0              0              0            0   
4                   0              1              0            0   
...               ...            ...            ...          ...   
16040               0              0              0            1   
16041               0              0              0            1   
16042               0              0              0            1   
16043               0              0              0            1   
16044               0              0              0            0   

       name_Documentary  name_Drama  name_Family  name_Foreign  name_Games  \
0                     0           0            1             0           0   
1                     0           0            0             0           0   
2                     0           0            0             0           0   
3                     0           0            0             0           0   
4                     0           0            0             0           0   
...                 ...         ...          ...           ...         ...   
16040                 0           0            0             0           0   
16041                 0           0            0             0           0   
16042                 0           0            0             0           0   
16043                 0           0            0             0           0   
16044                 1           0            0             0           0   

       name_Horror  name_Music  name_New  name_Sci-Fi  name_Sports  \
0                0           0         0            0            0   
1                0           1         0            0            0   
2                0           0         0            0            0   
3                1           0         0            0            0   
4                0           0         0            0            0   
...            ...         ...       ...          ...          ...   
16040            0           0         0     

In [10]:
data_cat

,name,title,description,rating
0,Family,BLANKET BEVERLY,A Emotional Documentary of a Student And a Gir...,G
1,Music,FREAKY POCUS,A Fast-Paced Documentary of a Pastry Chef And ...,R
2,Children,GRADUATE LORD,A Lacklusture Epistle of a Girl And a A Shark ...,G
3,Horror,LOVE SUICIDES,A Brilliant Panorama of a Hunter And a Explore...,R
4,Children,IDOLS SNATCHERS,A Insightful Drama of a Car And a Composer who...,NC-17
...,...,...,...,...
16040,Comedy,GROUNDHOG UNCUT,A Brilliant Panorama of a Astronaut And a Tech...,PG-13
16041,Comedy,GUNFIGHT MOON,A Epic Reflection of a Pastry Chef And a Explo...,NC-17
16042,Comedy,GUNFIGHT MOON,A Epic Reflection of a Pastry Chef And a Explo...,NC-17
16043,Comedy,MULAN MOON,A Emotional Saga of a Womanizer And a Pioneer ...,G


In [11]:
data_cat=data_cat.drop(['title','description'], axis=1)

In [12]:
data_cat

,name,rating
0,Family,G
1,Music,R
2,Children,G
3,Horror,R
4,Children,NC-17
...,...,...
16040,Comedy,PG-13
16041,Comedy,NC-17
16042,Comedy,NC-17
16043,Comedy,G


In [13]:
encoder = OneHotEncoder(handle_unknown='error', drop='first')
encoder.fit(data_cat[['name']])

OneHotEncoder(drop='first')

In [14]:
encoded = encoder.transform(data_cat[['name']]).toarray()
encoded #.shape # 

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [15]:
df = pd.DataFrame(encoded)
df.columns = encoder.categories_[0][1:]
df

,Animation,Children,Classics,Comedy,Documentary,Drama,Family,Foreign,Games,Horror,Music,New,Sci-Fi,Sports,Travel
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16040,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16041,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16042,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16043,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
encoder1 = OneHotEncoder(handle_unknown='error', drop='first')
encoder.fit(data_cat[['rating']])

OneHotEncoder(drop='first')

In [17]:
encoded1 = encoder.transform(data_cat[['rating']]).toarray()
encoded1 #.shape # 

array([[0., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       ...,
       [1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 1., 0., 0.]])

In [18]:
df1 = pd.DataFrame(encoded1)
df1.columns = encoder.categories_[0][1:]
df1

,NC-17,PG,PG-13,R
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0
...,...,...,...,...
16040,0.0,0.0,1.0,0.0
16041,1.0,0.0,0.0,0.0
16042,1.0,0.0,0.0,0.0
16043,0.0,0.0,0.0,0.0


In [19]:
bigdata = pd.concat([df, df1], axis =1)
bigdata

,Animation,Children,Classics,Comedy,Documentary,Drama,Family,Foreign,Games,Horror,Music,New,Sci-Fi,Sports,Travel,NC-17,PG,PG-13,R
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16040,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
16041,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
16042,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
16043,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
data_num

,rental_id,film_id,category_id,release_year,rental_duration,rental_rate,length
0,1,80,8,2006,7,2.99,148
1,2,333,12,2006,7,2.99,126
2,3,373,3,2006,7,2.99,156
3,4,535,11,2006,6,0.99,181
4,5,450,3,2006,5,2.99,84
...,...,...,...,...,...,...,...
16040,13780,385,5,2006,6,4.99,139
16041,11847,388,5,2006,5,0.99,70
16042,15423,388,5,2006,5,0.99,70
16043,15314,604,5,2006,4,0.99,160


In [21]:
data_num.dtypes

rental_id            int64
film_id              int64
category_id          int64
release_year         int64
rental_duration      int64
rental_rate        float64
length               int64
dtype: object

In [22]:
count_nan = data_num.isna().sum().sum()

print ('Count of NaN: ' + str(count_nan))

Count of NaN: 0


In [23]:
data1 = pd.concat([bigdata, data_num], axis =1)

In [24]:
cols_to_keep = ['rental_date']

In [25]:
data11 = data10.drop(data10.columns.difference(cols_to_keep), axis=1)

In [26]:
data11

,rental_date
0,2005-05-24 22:53:30
1,2005-05-24 22:54:33
2,2005-05-24 23:03:39
3,2005-05-24 23:04:41
4,2005-05-24 23:05:21
...,...
16040,2006-02-14 15:16:03
16041,2006-02-14 15:16:03
16042,2006-02-14 15:16:03
16043,2006-02-14 15:16:03


In [27]:
data20 = pd.concat([data11, data1], axis =1)

In [28]:
data20

,rental_date,Animation,Children,Classics,Comedy,Documentary,Drama,Family,Foreign,Games,Horror,Music,New,Sci-Fi,Sports,Travel,NC-17,PG,PG-13,R,rental_id,film_id,category_id,release_year,rental_duration,rental_rate,length
0,2005-05-24 22:53:30,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,80,8,2006,7,2.99,148
1,2005-05-24 22:54:33,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,333,12,2006,7,2.99,126
2,2005-05-24 23:03:39,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,373,3,2006,7,2.99,156
3,2005-05-24 23:04:41,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4,535,11,2006,6,0.99,181
4,2005-05-24 23:05:21,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5,450,3,2006,5,2.99,84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16040,2006-02-14 15:16:03,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,13780,385,5,2006,6,4.99,139
16041,2006-02-14 15:16:03,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,11847,388,5,2006,5,0.99,70
16042,2006-02-14 15:16:03,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,15423,388,5,2006,5,0.99,70
16043,2006-02-14 15:16:03,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15314,604,5,2006,4,0.99,160


In [29]:
df3 = data20.copy()
start_date = '2005-05-01'
end_date = '2005-06-01'
mask = (df3['rental_date'] >= start_date) & (df3['rental_date'] < end_date)
  
df3 = df3.loc[mask]
df3

,rental_date,Animation,Children,Classics,Comedy,Documentary,Drama,Family,Foreign,Games,Horror,Music,New,Sci-Fi,Sports,Travel,NC-17,PG,PG-13,R,rental_id,film_id,category_id,release_year,rental_duration,rental_rate,length
0,2005-05-24 22:53:30,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,80,8,2006,7,2.99,148
1,2005-05-24 22:54:33,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,333,12,2006,7,2.99,126
2,2005-05-24 23:03:39,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,373,3,2006,7,2.99,156
3,2005-05-24 23:04:41,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4,535,11,2006,6,0.99,181
4,2005-05-24 23:05:21,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5,450,3,2006,5,2.99,84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1151,2005-05-31 21:36:44,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1153,598,15,2006,6,0.99,57
1152,2005-05-31 21:42:09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1154,599,15,2006,3,0.99,103
1153,2005-05-31 22:17:11,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1155,445,4,2006,5,2.99,100
1154,2005-05-31 22:37:34,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1156,101,6,2006,3,0.99,73


In [30]:
data20

df2 = data20.copy()
start_date = '2005-08-01'
end_date = '2005-09-01'
mask = (df2['rental_date'] >= start_date) & (df2['rental_date'] < end_date)
  
df2 = df2.loc[mask]
df2

,rental_date,Animation,Children,Classics,Comedy,Documentary,Drama,Family,Foreign,Games,Horror,Music,New,Sci-Fi,Sports,Travel,NC-17,PG,PG-13,R,rental_id,film_id,category_id,release_year,rental_duration,rental_rate,length
10176,2005-08-01 00:00:44,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,10181,683,7,2006,5,4.99,60
10177,2005-08-01 00:08:01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,10183,941,10,2006,4,4.99,145
10178,2005-08-01 00:08:01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,10182,894,16,2006,6,0.99,68
10179,2005-08-01 00:09:33,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,10184,611,4,2006,7,4.99,73
10180,2005-08-01 00:12:11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10185,281,10,2006,6,0.99,143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15857,2005-08-23 22:25:26,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,16045,168,3,2006,5,0.99,67
15858,2005-08-23 22:26:47,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,16046,951,4,2006,6,0.99,78
15859,2005-08-23 22:42:48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,16047,452,9,2006,4,0.99,122
15860,2005-08-23 22:43:07,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,16048,439,7,2006,4,4.99,151


In [31]:
df2['targets'] = df2['rental_date'].apply(lambda x: 0 if x == '2005-08-01' else 1)

In [32]:
df2

,rental_date,Animation,Children,Classics,Comedy,Documentary,Drama,Family,Foreign,Games,Horror,Music,New,Sci-Fi,Sports,Travel,NC-17,PG,PG-13,R,rental_id,film_id,category_id,release_year,rental_duration,rental_rate,length,targets
10176,2005-08-01 00:00:44,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,10181,683,7,2006,5,4.99,60,1
10177,2005-08-01 00:08:01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,10183,941,10,2006,4,4.99,145,1
10178,2005-08-01 00:08:01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,10182,894,16,2006,6,0.99,68,1
10179,2005-08-01 00:09:33,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,10184,611,4,2006,7,4.99,73,1
10180,2005-08-01 00:12:11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10185,281,10,2006,6,0.99,143,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15857,2005-08-23 22:25:26,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,16045,168,3,2006,5,0.99,67,1
15858,2005-08-23 22:26:47,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,16046,951,4,2006,6,0.99,78,1
15859,2005-08-23 22:42:48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,16047,452,9,2006,4,0.99,122,1
15860,2005-08-23 22:43:07,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,16048,439,7,2006,4,4.99,151,1


In [33]:
df2['targets']

10176    1
10177    1
10178    1
10179    1
10180    1
        ..
15857    1
15858    1
15859    1
15860    1
15861    1
Name: targets, Length: 5686, dtype: int64

In [34]:
count_nan = df2['targets'].isna().sum().sum()

print ('Count of NaN: ' + str(count_nan))

Count of NaN: 0


In [35]:
df3['targets'] = df3['rental_date'].apply(lambda x: 1 if x == '2005-08-01' else 0)

In [36]:
df3

,rental_date,Animation,Children,Classics,Comedy,Documentary,Drama,Family,Foreign,Games,Horror,Music,New,Sci-Fi,Sports,Travel,NC-17,PG,PG-13,R,rental_id,film_id,category_id,release_year,rental_duration,rental_rate,length,targets
0,2005-05-24 22:53:30,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,80,8,2006,7,2.99,148,0
1,2005-05-24 22:54:33,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,333,12,2006,7,2.99,126,0
2,2005-05-24 23:03:39,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,373,3,2006,7,2.99,156,0
3,2005-05-24 23:04:41,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4,535,11,2006,6,0.99,181,0
4,2005-05-24 23:05:21,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5,450,3,2006,5,2.99,84,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1151,2005-05-31 21:36:44,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1153,598,15,2006,6,0.99,57,0
1152,2005-05-31 21:42:09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1154,599,15,2006,3,0.99,103,0
1153,2005-05-31 22:17:11,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1155,445,4,2006,5,2.99,100,0
1154,2005-05-31 22:37:34,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1156,101,6,2006,3,0.99,73,0


In [37]:
df3['targets']

0       0
1       0
2       0
3       0
4       0
       ..
1151    0
1152    0
1153    0
1154    0
1155    0
Name: targets, Length: 1156, dtype: int64

In [38]:
df3['targets'].value_counts()

0    1156
Name: targets, dtype: int64

In [39]:
df2['targets'].value_counts()

1    5686
Name: targets, dtype: int64

In [40]:
df5 = pd.concat([df2, df3])

In [41]:
df5

,rental_date,Animation,Children,Classics,Comedy,Documentary,Drama,Family,Foreign,Games,Horror,Music,New,Sci-Fi,Sports,Travel,NC-17,PG,PG-13,R,rental_id,film_id,category_id,release_year,rental_duration,rental_rate,length,targets
10176,2005-08-01 00:00:44,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,10181,683,7,2006,5,4.99,60,1
10177,2005-08-01 00:08:01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,10183,941,10,2006,4,4.99,145,1
10178,2005-08-01 00:08:01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,10182,894,16,2006,6,0.99,68,1
10179,2005-08-01 00:09:33,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,10184,611,4,2006,7,4.99,73,1
10180,2005-08-01 00:12:11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10185,281,10,2006,6,0.99,143,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1151,2005-05-31 21:36:44,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1153,598,15,2006,6,0.99,57,0
1152,2005-05-31 21:42:09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1154,599,15,2006,3,0.99,103,0
1153,2005-05-31 22:17:11,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1155,445,4,2006,5,2.99,100,0
1154,2005-05-31 22:37:34,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1156,101,6,2006,3,0.99,73,0


In [42]:
df5['targets'].value_counts()

1    5686
0    1156
Name: targets, dtype: int64

In [43]:
with_film_id = df5.set_index('film_id')

In [44]:
df5.drop(['rental_id','rental_date','category_id','rental_id','release_year'], axis=1)

,Animation,Children,Classics,Comedy,Documentary,Drama,Family,Foreign,Games,Horror,Music,New,Sci-Fi,Sports,Travel,NC-17,PG,PG-13,R,film_id,rental_duration,rental_rate,length,targets
10176,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,683,5,4.99,60,1
10177,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,941,4,4.99,145,1
10178,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,894,6,0.99,68,1
10179,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,611,7,4.99,73,1
10180,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,281,6,0.99,143,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1151,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,598,6,0.99,57,0
1152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,599,3,0.99,103,0
1153,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,445,5,2.99,100,0
1154,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,101,3,0.99,73,0


In [45]:
with_film_id2 = df5.set_index('film_id')

In [46]:
df5 = df5.drop(['rental_id','rental_date','category_id','rental_id','release_year','film_id'], axis=1)

In [47]:
X = df5.drop('targets', axis=1)
y = df5['targets']

In [50]:
X

,Animation,Children,Classics,Comedy,Documentary,Drama,Family,Foreign,Games,Horror,Music,New,Sci-Fi,Sports,Travel,NC-17,PG,PG-13,R,rental_duration,rental_rate,length
10176,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5,4.99,60
10177,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4,4.99,145
10178,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,6,0.99,68
10179,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,7,4.99,73
10180,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,0.99,143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1151,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6,0.99,57
1152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,3,0.99,103
1153,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,2.99,100
1154,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3,0.99,73


In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [53]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [54]:
classification = LogisticRegression(random_state=42, max_iter=10000)
classification.fit(X_train, y_train)

LogisticRegression(max_iter=10000, random_state=42)

In [55]:
classification.score(X_test, y_test)

0.8385682980277575

In [63]:
predictions = classification.predict(X_test)
confusion_matrix(y_test, predictions)

array([[   0,  221],
       [   0, 1148]])

In [66]:
print(metrics.classification_report(y_test, predictions))

NameError: name 'metrics' is not defined